In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
import time
import lxml
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import requests
import os

### Login + Page

In [3]:
# 抓取貼文內容--------------------
browser = webdriver.Chrome()
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("14han_15felix_")
password_input.send_keys("Yaya0329@Instagram")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(20)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['wendys', 'sonicdrivein', 'mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']

for subUrl in subUrlList:
    # 前往頁面
    browser.get(url + '/' + subUrl)
    time.sleep(2)
    # 往下滑並取得新的貼文連結
    post_all = []
    postList = []

    # Get scroll height.
    last_height = browser.execute_script("return document.body.scrollHeight")
    with tqdm(total=100) as pbar:
        while True:
            # Scroll down to the bottom.
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(5)
    
            soup = Soup(browser.page_source, 'lxml')
            # 尋找所有的貼文連結
        
            for rowPost in soup.select('div._ac7v'):
                for elem in rowPost.select('div a'):
                    if elem['href'] in postList:
                        continue
                    postList.append(elem['href'])
                    img_elem = elem.select('div div img')
                    temp = []
                    temp.append(elem['href'])
                    try: 
                        temp.append(img_elem[0]['alt'])
                    except:
                        temp.append("")
                    temp.append(img_elem[0]['src'])
                    post_all.append(temp)
                    
            # Calculate new scroll height and compare with last scroll height.
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            pbar.update(1)
        
    print(subUrl+" 總共 " + str(len(post_all)) + " 篇貼文")
    
    # 存成csv
    postDataframe= pd.DataFrame(post_all)
    postDataframe.rename(columns = {0:'url', 1:'imgAlt', 2:'imgSrc'}, inplace = True)
    postDataframe.to_csv('Home_'+subUrl+'.csv', index = False)

inputing username and password...
login successfully


 30%|███       | 30/100 [02:37<06:07,  5.25s/it]


wendys 總共 371 篇貼文


186it [15:51,  5.11s/it]                         


sonicdrivein 總共 2240 篇貼文


...

mcdonalds 總共 307 篇貼文


173it [14:45,  5.12s/it]                         


mcdonalds_switzerland 總共 2087 篇貼文


 70%|███████   | 70/100 [06:01<02:34,  5.16s/it]

mcdonaldscanada 總共 850 篇貼文


In [2]:
# 抓取貼文內容--------------------
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=334,778")
browser = webdriver.Chrome(options=chrome_options)
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_blood_mango_")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(15)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['sonicdrivein']
# 'wendys', 'mcdonalds', 'mcdonalds_switzerland', 'mcdonaldscanada', 'mcdonalds_switzerland'
# ,'sonicdrivein'

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    # insHome = pd.read_csv('Home_'+subUrl+'.csv')
    insHome = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insHome.shape[0]) as pbar:
        counter = 0
        for index, row in insHome.iterrows(): #< 922
            if '/p/' in row['url'] and counter > 406 and counter < 923:
                browser.get(url + row['url'])
                time.sleep(10)
                html = browser.page_source
                soup = Soup(html, 'lxml')
                img_elem = soup.select('img.x5yr21d')
                text_elem = soup.select('h1._ap3a')

                insHome.at[index, 'imgSrc'] = img_elem[1]['src']
                try:
                    insHome.at[index, 'imgAlt'] = text_elem[0].text
                except:
                    insHome.at[index, 'imgAlt'] = ""
                insHome.to_csv('Done_'+subUrl+'.csv', index = False)
            pbar.update(1)
            counter = counter+1
            
    insHome['imgName'] = subUrl+'_'+insHome.index.astype(str)
    insHome.to_csv('Done_'+subUrl+'.csv', index = False)

inputing username and password...
login successfully
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


100%|██████████| 2087/2087 [1:20:06<00:00,  2.30s/it] 


### Download Image

In [3]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']
#  'sonicdrivein', 
for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insDone.shape[0]) as pbar:
        for index, row in insDone.iterrows():
            #create img folder
            if not os.path.exists(subUrl+'_img'):
                os.makedirs(subUrl+'_img')
            with open(subUrl+'_img/'+row['imgName']+'.jpg', 'wb') as f:
                try :
                    f.write(requests.get(row['imgSrc']).content)
                except:
                    print('Failed to download' + row['imgName']+'.jpg')
            pbar.update(1)

parsing wendys.csv
parsing wendys.csv done


100%|██████████| 371/371 [00:15<00:00, 23.50it/s]


parsing mcdonalds.csv
parsing mcdonalds.csv done


100%|██████████| 307/307 [00:20<00:00, 14.79it/s]


parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


100%|██████████| 2087/2087 [01:35<00:00, 21.76it/s]


parsing mcdonaldscanada.csv
parsing mcdonaldscanada.csv done


100%|██████████| 850/850 [00:31<00:00, 26.81it/s]


In [4]:
subUrlList = ['sonicdrivein']
#  'sonicdrivein', 
for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insDone.shape[0]) as pbar:
        for index, row in insDone.iterrows():
            #create img folder
            if not os.path.exists(subUrl+'_img'):
                os.makedirs(subUrl+'_img')
            with open(subUrl+'_img/'+row['imgName']+'.jpg', 'wb') as f:
                try :
                    f.write(requests.get(row['imgSrc']).content)
                except:
                    print('Failed to download' + row['imgName']+'.jpg')
            pbar.update(1)

parsing sonicdrivein.csv
parsing sonicdrivein.csv done


100%|██████████| 2240/2240 [14:36<00:00,  2.56it/s] 
